In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
from glob import glob
from subprocess import check_call, check_output
import textwrap

import pandas as pd

import oae_smyle
import project

In [3]:
exclude = []
if os.path.exists("case-info/case-info.submitted"):
    ex = pd.read_csv("case-info.submitted", header=None)
    for row in ex.iterrows():
        exclude.append((row[1][0], row[1][1], row[1][2]))
exclude

[]

In [4]:
df = pd.read_csv("oae-simulations.csv", index_col=("polygon", "basin", "start_date")).drop(columns=["index", "rmse"])
for x in exclude:
    df = df.drop(x)

df

,,,polygon_id
polygon,basin,start_date,
0,North_Atlantic_basin,0347-04,0
1,North_Atlantic_basin,0347-04,1
2,North_Atlantic_basin,0347-04,2
3,North_Atlantic_basin,0347-04,3
4,North_Atlantic_basin,0347-04,4
...,...,...,...
35,Southern_Ocean,0347-10,685
36,Southern_Ocean,0347-10,686
37,Southern_Ocean,0347-10,687


In [5]:
files = []
for row in df.iterrows():
    p = f"{row[0][0]:03d}"
    b = row[0][1]
    d = row[0][2].replace("0347", "1999")
    file = f"{project.dir_data}/alk-forcing/OAE-Efficiency-Map/alk-forcing-{b}.{p}-{d}.nc"
    assert os.path.exists(file), file
    files.append(file)
    
df["file"] = files
df

,,,polygon_id,file
polygon,basin,start_date,,
0,North_Atlantic_basin,0347-04,0,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
1,North_Atlantic_basin,0347-04,1,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
2,North_Atlantic_basin,0347-04,2,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
3,North_Atlantic_basin,0347-04,3,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
4,North_Atlantic_basin,0347-04,4,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
...,...,...,...,...
35,Southern_Ocean,0347-10,685,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
36,Southern_Ocean,0347-10,686,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...
37,Southern_Ocean,0347-10,687,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...


In [ ]:
n_run_cases = 1000
attempt = "001"

os.makedirs("case-info", exist_ok=True)
os.makedirs("case-info/build-in", exist_ok=True)
os.makedirs("case-info/build-out", exist_ok=True)

def submission_script(case, alk_forcing_file, refdate):
    header = textwrap.dedent(
        f"""\
    #!/bin/bash
    #SBATCH -J build.{case}
    #SBATCH -A {project.account}
    #SBATCH -e case-info/build-out/{case}-%J.out
    #SBATCH -o case-info/build-out/{case}-%J.out
    #SBATCH --time=01:00:00
    #SBATCH --mem=12GB
    #SBATCH --qos=shared
    #SBATCH --nodes=1
    #SBATCH --ntasks=1
    #SBATCH --constraint=cpu

    module purge
    module restore
    module load conda
    conda activate cworthy
    
    python oae_smyle.py --case {case} --alk-forcing-file {alk_forcing_file} --refdate {refdate}

    """
    )
    
    build_script = f"case-info/build-in/{case}.build"
    with open(build_script, "w") as fid:
        fid.write(header)
    
    return build_script


build_scripts = []
jobid = []
for row in df.iterrows():
    polygon_ndx = f"{row[0][0]:03d}"
    basin = row[0][1]
    refdate = f"{row[0][2]}-01"
    alk_forcing_file = row[1]["file"]
    
    case = f"smyle.oae-map.{basin}.{polygon_ndx}.{refdate.replace('0347', '1999')}.{attempt}"

    build_script = submission_script(case, alk_forcing_file, refdate)
    
    slurm_str = check_output(["sbatch", build_script]).decode('UTF-8').strip()
    jobid.append(slurm_str.split(" ")[-1])
    
    with open("case-info/case-info.built", "a") as fid:
        fid.write(f"{row[0][0]},{row[0][1]},{row[0][2]}\n")
        
    build_scripts.append(build_scripts)   
    if len(build_scripts) == n_run_cases:
        break
    
build_scripts            

In [98]:
cases = sorted([f for f in os.listdir(project.dir_caseroot_root) if "smyle.oae-map." in f])

rows = []
df_caseinfo = None

for case in cases:
    CaseStatus = f"{project.dir_caseroot_root}/{case}/CaseStatus"
    
    with open(CaseStatus, "r") as fid:
        lines = fid.readlines()
    
    row_data = dict(
        case=case,
        build=any("case.build success" in line for line in lines),
        submitted=any("case.submit success" in line for line in lines),
        running=any("case.run starting" in line for line in lines),
        run=any("case.run success" in line for line in lines),
        archive=any("st_archive success" in line for line in lines),
        error=any("ERROR" in line for line in lines),
    )
    rows.append(row_data)

if rows:
    df_caseinfo = pd.DataFrame(rows).set_index("case")
df_caseinfo

,build,submitted,running,run,archive,error
case,,,,,,
smyle.oae-map.North_Atlantic_basin.000.1999-04-01.001,True,True,True,False,False,False
smyle.oae-map.North_Atlantic_basin.000.1999-07-01.001,True,True,True,False,False,False
smyle.oae-map.North_Atlantic_basin.000.1999-10-01.001,True,True,True,False,False,False
smyle.oae-map.North_Atlantic_basin.001.1999-04-01.001,True,True,True,False,False,False
smyle.oae-map.North_Atlantic_basin.001.1999-07-01.001,True,True,True,False,False,False
...,...,...,...,...,...,...
smyle.oae-map.Southern_Ocean.035.1999-10-01.001,True,True,False,False,False,False
smyle.oae-map.Southern_Ocean.036.1999-10-01.001,True,True,False,False,False,False
smyle.oae-map.Southern_Ocean.037.1999-10-01.001,True,True,False,False,False,False


In [ ]:
cases = df_caseinfo.loc[(~df_caseinfo.error) & (df_caseinfo.build) & (~df_caseinfo.submitted)].index.to_list()
cases

In [97]:
os.makedirs("case-info/submit-out", exist_ok=True)

cases = df_caseinfo.loc[(~df_caseinfo.error) & (df_caseinfo.build) & (~df_caseinfo.submitted)].index.to_list()

n_simult = 10
shebang = textwrap.dedent("""#!/bin/bash
set -e

module load python

i=0
pids=()
"""
                         )
# pids=()
# for i in $n_procs; do
#     ./procs[${i}] &
#     pids[${i}]=$!
# done

# # wait for all pids
# for pid in ${pids[*]}; do
#     wait $pid
# done



script = [shebang]
submitted = []
n_sub = 0
for n, case in enumerate(cases):
    script.append(textwrap.dedent(
        f"""
        cd {project.dir_caseroot_root}/{case}q
        ./case.submit &> {project.scriptroot}/case-info/submit-out/{case}.submit &
        
        pids[${{i}}]=$!
        i=$(($i+1))
        
        """
    ))
    
    if (len(script) - 1 == n_simult) or n + 1 == len(cases):
        script.append(textwrap.dedent(
        """        
        for pid in ${pids[*]}; do
            echo "waiting on ${pid}"
            wait $pid
        done
        """
        ))
        with open("case-info/batch.case.submit", "w") as fid:
            fid.writelines(script)
        n_sub += len(script) - 1
        
        check_call("bash batch.case.submit > out", shell=True, cwd="case-info")
        script = [shebang]
        
assert n_sub == len(cases)

AssertionError: 

In [ ]:
df_caseinfo[df_caseinfo.running]

In [31]:
len(cases)

738